# Notebook for EAGLE.lib modules development
This notebook should be run from test environvent (not development)

In [3]:
# imports
import numpy
import scipy

from EAGLE.lib.seqs import load_fasta_to_dict

## Conservative columns irregularity

In [ ]:
def estimate_irregularity(mult_aln_dict, cons_thr=0.7, window_l=50, windows_step=25):  # This function will be the MultAln classmethod
    # mult_aln_dict will be replaced with self.mult_aln_dict
    windows_list = list()
    i = 0
    while i < (len(mult_aln_dict[mult_aln_dict.keys()[0]])):
        windows_list.append(mult_aln_dict[i: i+window_l])#This does not work - it is only the idea
        i += windows_step
    cons_cols_by_windows = numpy.array(map(, windows_list))
    return scipy.chisquere